# Compare Sentinel-2 and Venus


 This notebook demonstrates querying the STAC API for a Venus and Sentinel-2 chip using an ROI, calculate the NDVI and plot it as a comparison

## Import and Init Environment

In [1]:
# Import

from glob import glob
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pystac_client import Client
from matplotlib import pyplot as plt
import geopandas as gpd
import requests
import seaborn as sns
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from odc.stac import configure_rio, stac_load

ModuleNotFoundError: No module named 'dotenv'

Authenticate with the STAC API, if you get an error check your credentials in `.env`

In [ ]:
os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'
load_dotenv()  # take environment variables from .env.

def get_new_token():
    auth_server_url = os.getenv("EDS_AUTH_URL")
    client_id = os.getenv("EDS_CLIENT_ID")
    client_secret = os.getenv("EDS_SECRET")
    token_req_payload = {'grant_type': 'client_credentials'}

    token_response = requests.post(
        auth_server_url,
        data=token_req_payload,
        verify=False,
        allow_redirects=False,
        auth=(client_id, client_secret)
    )
    token_response.raise_for_status()

    tokens = json.loads(token_response.text)
    return tokens['access_token']

token = get_new_token()

catalog = Client.open(os.getenv("EDS_API_URL"), headers={
    "Authorization": f"bearer {token}"
})

In [ ]:
## Set the mission and the collection

In [ ]:
SATELLITE = "S2" # "S2" or "VENUS"
COLLECTION = "sentinel-2-l2a"

In [ ]:
## Bounding Box

In [ ]:
def get_bounds_polygon_dict() -> tuple:
    #gdf = gpd.GeoDataFrame.from_features([polygon_dict])
    gdf = gpd.read_file("./stac-notebooks/pivot_corumba.geojson")
    polygon = gdf.geometry.iloc[0]
    bounds = polygon.bounds
    return bounds

bbox = get_bounds_polygon_dict()

## Define the config

In [ ]:
start_date = "2019-11-01"
end_date = "2020-05-01"
max_clouds = 30 # the maximum cloud cover percentage. Note this is over the WHOLE image, not just the ROI

In [ ]:
## Query the collection items

In [ ]:
query = catalog.search(
    collections=[COLLECTION], 
    datetime=f"{start_date}/{end_date}",
    bbox=bbox,
    query={"eda:ag_cloud_mask_available":{"eq":True}, "eo:cloud_cover":{"lt":max_clouds}},
)

items = list(query.get_items())
print(f"Found: {len(items):d} datasets


## Generate and load the Sentinel-2 datacube and calculate NDVI

In [ ]:
# Sentinel-2 datacube
dataset = stac_load(
    items,
    bands=("red", "green", "blue","nir"),
    crs="epsg:3857", # since resolution is in metres, we need to use a projected CRS
    resolution=5, # the resolution of the output image in metres
    # chunks={},  # Uncomment if using dask cluster
    groupby="id",
    bbox=bbox,
)

print(dataset)
dataset.attrs['sensor'] = "Sentinel-2-l2a"

# NDVI plot

meanval = dataset.mean(('x','y')) #.ed.add_indices(['NDVI'])['NDVI'].plot(ls='-',marker='o', label='Sentinel-2')
meanval.attrs['sensor'] = "Sentinel-2-l2a"

ndvixy = (meanval["nir"] - meanval["red"]) / (meanval["nir"] + meanval["red"])
meanval["ndvi"] = ndvixy

#dataset = dataset.load()
meanval = meanval.load()

print(meanval)


## Generate and load the Venus datacube

In [ ]:
COLLECTION = "venus-l2a"
#    BANDS = ("red", "green", "blue", ) # "nir08", "rededge", "yellow", "coastal"

query = catalog.search(
    collections=[COLLECTION], 
    datetime=f"{start_date}/{end_date}",
    bbox=bbox,
    query={"eo:cloud_cover":{"lt":max_clouds}},
)

items = list(query.get_items())
print(f"Found: {len(items):d} datasets")

datasetv = stac_load(
    items,
    bands=("red", "green", "blue","nir08", "detailed_cloud_mask"),
    crs="epsg:3857", # since resolution is in metres, we need to use a projected CRS
    resolution=5, # the resolution of the output image in metres
    # chunks={},  # Uncomment if using dask cluster
    # geobox=dataset.odc.geobox,
    groupby="id",
    bbox=bbox,
)

print(datasetv)
datasetv.attrs['sensor'] = "Venus-l2a"

# NDVI plot
meanvalV = datasetv.mean(('x','y')) #.ed.add_indices(['NDVI'])['NDVI'].plot(ls='-',marker='o', label='Sentinel-2')
meanvalV.attrs['sensor'] = "Venus-l2a"

ndvixyV = (meanvalV["nir08"] - meanvalV["red"]) / (meanvalV["nir08"] + meanvalV["red"])
meanvalV["ndvi"] = ndvixyV

#dataset = dataset.load()
meanvalV = meanvalV.load()
print (meanvalV)


# Results (plots)

## Plot and Compare the NDVI

In [ ]:

plt.title('NDVI between Sentinel-2 and Venus')

#for sensor in [meanval, meanvalV]:
meanval["ndvi"].plot()
plt.legend("STentinel 2")
#plt.suptitle("Sentinel 2")

meanvalV["ndvi"].plot()
#plt.legend("Venus")
#plt.suptitle("Venus")
    
plt.show()